In [1]:
!pip install --upgrade pip

In [2]:
!pip install pytesseract
!pip install opencv-python
!pip install python-multipart pillow pytesseract opencv-python numpy uvicorn

In [3]:
!source ../.venv/bin/activate

In [5]:
import pytesseract
import argparse
import cv2

In [6]:
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True,
	help="path to input image to be OCR'd")
args = vars(ap.parse_args())

usage: ipykernel_launcher.py [-h] -i IMAGE
ipykernel_launcher.py: error: the following arguments are required: -i/--image


SystemExit: 2

/opt/homebrew/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
image = cv2.imread(args["image"])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# use Tesseract to OCR the image
text = pytesseract.image_to_string(image)
print(text)

NameError: name 'args' is not defined